In [1]:
import os
import yaml
import pytest
import pandas as pd

from network_wrangler import ProjectCard

from update_registry import update_registry
from methods_io import read_project_cards
from methods_add_cards import add_cards_to_registry
from methods_add_cards import _make_available

## defaults

In [2]:
CARD_DIR = os.path.join(".", "reference_projects")
REGISTRY_FILE = "registry.csv"
CONFIG_FILE = "registry_config.yml"

In [ ]:
input_reg_df = pd.read_csv(REGISTRY_FILE)
input_reg_df

## test methods

In [ ]:
card_file_list = read_project_cards(CARD_DIR)
card_file_list

In [ ]:
update_registry(
        CONFIG_FILE, REGISTRY_FILE, "delete_me.csv", CARD_DIR, False,
    )

## test 01

In [ ]:
input_file = "test_input_registry.csv"
output_file = "test_update_registry.csv"

data = []
input_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
input_df.to_csv(input_file, index=False)

update_registry(
        config_file="registry_config.yml",
        input_reg_file=input_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "tests", "projects", "project_AB"),
        write_card_updates=False,
)

data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
]
    
target_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_df = target_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

# os.remove(input_file)
# os.remove(output_file)

In [ ]:
target_df

In [ ]:
outcome_df

In [ ]:
target_df.equals(outcome_df)

## test 02

In [ ]:
output_file = "test_update_registry.csv"
temp_file = "temp_registry.csv"
data = [
        ["node", 1001, "Project Z"],
        ["node", 1002, "Project Z"],
        ["link", 501, "Project Z"],
    ]
temp_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
temp_df.to_csv(temp_file, index=False)

update_registry(
        config_file="registry_config.yml",
        input_reg_file=temp_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "tests", "projects", "project_AB"),
        write_card_updates=False,
    )

data = [
        ["node", 1001, "Project Z"],
        ["node", 1002, "Project Z"],
        ["link", 501, "Project Z"],
        ["node", 1003, "Project B"],
        ["node", 1004, "Project B"],
        ["node", 1005, "Project A"],
        ["node", 1006, "Project A"],
        ["link", 502, "Project B"],
        ["link", 503, "Project A"],
    ]
target_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_df = target_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

In [ ]:
target_df

In [ ]:
outcome_df

In [ ]:
target_df.equals(outcome_df)

## test 03
make sure procedures are not recursive. if project name is in the repository, take no action.

In [ ]:
input_file = "test_input_registry.csv"
output_file = "test_update_registry.csv"

data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
    ]
input_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
input_df.to_csv(input_file)

update_registry(
        config_file="registry_config.yml",
        input_reg_file=input_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "tests", "projects", "project_AB"),
        write_card_updates=False,
)

data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
]
    
target_i_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_i_df = target_i_df.sort_values(by=["type", "id"]).reset_index(drop=True)

data = [
        ["node", 1001, "Project A"],
        ["node", 1002, "Project A"],
        ["node", 1003, "Project B"],
        ["node", 1004, "Project B"],
        ["link", 501, "Project A"],
        ["link", 502, "Project B"],
]
    
target_ii_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_ii_df = target_ii_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df[["type", "id", "project_added"]].sort_values(by=["type", "id"]).reset_index(drop=True)

os.remove(input_file)
os.remove(output_file)


In [3]:
card_file = "./projects/project_B.yml"
card = ProjectCard.read(card_file, validate=False)

In [4]:
card_dict = card.__dict__
card_dict

{'project': 'Project B',
 'tags': ['registry_example'],
 'dependencies': '',
 'changes': [{'category': 'Roadway Deletion',
   'links': {'model_link_id': [123]}},
  {'category': 'Add New Roadway',
   'links': [{'A': 1001,
     'B': 1002,
     'model_link_id': 501,
     'distance': 0.25,
     'name': 'Broad Street',
     'lanes': 2,
     'bike_access': 1,
     'drive_access': 1,
     'transit_access': 0,
     'walk_access': 1}],
   'nodes': [{'X': -93.2222,
     'Y': 44.1111,
     'bike_node': 0,
     'drive_node': 1,
     'transit_node': 0,
     'walk_node': 0,
     'model_node_id': 1001},
    {'X': -93.3333,
     'Y': 44.2222,
     'bike_node': 0,
     'drive_node': 1,
     'transit_node': 0,
     'walk_node': 0,
     'model_node_id': 1002}]}],
 'file': './projects/project_B.yml',
 'valid': False}

In [5]:
temp_output = "delete_card.yml"
card.__dict__.pop("file")
card.__dict__.pop("valid")
card.__dict__

{'project': 'Project B',
 'tags': ['registry_example'],
 'dependencies': '',
 'changes': [{'category': 'Roadway Deletion',
   'links': {'model_link_id': [123]}},
  {'category': 'Add New Roadway',
   'links': [{'A': 1001,
     'B': 1002,
     'model_link_id': 501,
     'distance': 0.25,
     'name': 'Broad Street',
     'lanes': 2,
     'bike_access': 1,
     'drive_access': 1,
     'transit_access': 0,
     'walk_access': 1}],
   'nodes': [{'X': -93.2222,
     'Y': 44.1111,
     'bike_node': 0,
     'drive_node': 1,
     'transit_node': 0,
     'walk_node': 0,
     'model_node_id': 1001},
    {'X': -93.3333,
     'Y': 44.2222,
     'bike_node': 0,
     'drive_node': 1,
     'transit_node': 0,
     'walk_node': 0,
     'model_node_id': 1002}]}]}

In [ ]:
card_from_disk = ProjectCard.read(temp_output)
card_from_disk.__dict__

In [ ]:
for change_index, change_dict in enumerate(card_dict["changes"]):
    change_dict.update(change_dict)
    card_dict[change_index] = change_dict

In [ ]:
card.project

In [ ]:
card.changes

In [ ]:
change_dict = card.changes[1]
change_dict